# Mô hình gợi ý phim dựa trên nội dung.

* Ở phần trước chúng ta đã tìm hiểu về tệp dữ liệu, các feature, và cũng đã tìm hiểu về đánh giá bộ phim theo thể loại bằng trọng số điểm đánh giá theo 1 công thức.
* Tuy nhiên, ở phần này, chúng ta sẽ xây dựng một mô hình cụ thể hơn, sử dụng cosine_similarity để so sánh "độ gần gũi" (distance) của nó trên ma trận.
* **Tóm lại:** Ở đây, sẽ là bắt đầu của bài xây dựng model gợi ý phim, nhưng ở phần đầu này, chúng ta sẽ xây dựng một model không train (nói cách khác, có thể coi là 1 công thức đơn giản) để đưa ra gợi ý, tuy nhiên, nó vẫn đóng góp 1 phần lớn cho xây dựng mô hình gợi ý của chúng bằng __SVD__ trong thư viện __surprise__

Ở phần đầu tiên này, chúng ta sẽ đi từ từ các bước, tìm hiểu feature có tính tác động cao, loại bỏ các feature không cần thiết khỏi dataframe, và đưa ra những tham số thủ công, đánh giá bằng kinh nghiệm và kiến thức cá nhân để cải thiên model, bởi vì model này không có các độ đo để đánh giá độ chính xác hay độ lệch. (accuracy, mae, etc)

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')

In [2]:
links_small = pd.read_csv('input_data/links_small.csv')
md = pd.read_csv('input_data/movies_metadata.csv')

In [3]:
# Tiền xử lý dữ liệu: lấy ra cột imdbId vì chúng ta sẽ sử dụng Id phim dựa trên tmdb
# Chỉ lấy giá trị không null
links_small = links_small[links_small['tmdbId'].notnull()]
links_small.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9112 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9112 non-null   int64  
 1   imdbId   9112 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 284.8 KB


In [4]:
# convert sang int32 để đưa về 1 chuẩn chung, đang là float64
links_small= links_small['tmdbId'].astype('int')


In [5]:
links_small.info()

<class 'pandas.core.series.Series'>
Index: 9112 entries, 0 to 9124
Series name: tmdbId
Non-Null Count  Dtype
--------------  -----
9112 non-null   int32
dtypes: int32(1)
memory usage: 106.8 KB


In [6]:
## Hàm để tiền xử lý

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan


In [7]:
#chuyển id về int32 để làm chuẩn
#Tìm kiếm id null rồi loại bỏ
md['id'] = md['id'].apply(convert_int)
md[md['id'].isnull()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",NaN,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",NaN,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",NaN,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
md = md.drop([19730, 29503, 35587])

## Trong tệp dữ liệu metadata về movie thì có 45000 phim, tuy nhiên chúng ta sẽ thực hiện mô hình của chúng ta chỉ trên những phim có trong links_small

**Giải thích về điều này:**
* Tệp dữ liệu 45k rất tạp nham, hơn những thế, có rất nhiều phim không bao giờ được biết đến, trong khi các hệ thống streaming phim hiện nay như netflix hoặc bất kì hệ thống nào, để có cơ chế mua bản quyền, những phim không đáng giá sẽ không bao giờ xuất hiện ở trên những hệ thống này, chính vì thế, tệp 9000 mục đích được tạo ra là đã có chọn lọc những phim đáng giá, đáng để đưa vào mô hình.
* Mô hình của chúng ta ở bước hiện hiện tại (gợi ý phim dựa trên nội dung) là một mô hình không thông minh, nên chúng ta sẽ ưu tiên chất lượng trên hết.
* **Kết luận:** ÍT NHẤT, ở mô hình này, chúng ta sẽ chỉ sử dụng tệp dữ liệu 9000 phim, và chúng ta sẽ thực hiện bước xử lý để lấy nó ra khỏi tệp dữ liệu lớn __movies_metadata.csv__


In [9]:
md['id'] = md['id'].astype('int')

In [10]:
# tạo dataframe mới để phục vụ cho model
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 24)

In [11]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9099 entries, 0 to 45265
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  9099 non-null   object 
 1   belongs_to_collection  1674 non-null   object 
 2   budget                 9099 non-null   object 
 3   genres                 9099 non-null   object 
 4   homepage               1974 non-null   object 
 5   id                     9099 non-null   int32  
 6   imdb_id                9099 non-null   object 
 7   original_language      9099 non-null   object 
 8   original_title         9099 non-null   object 
 9   overview               9087 non-null   object 
 10  popularity             9099 non-null   object 
 11  poster_path            9096 non-null   object 
 12  production_companies   9099 non-null   object 
 13  production_countries   9099 non-null   object 
 14  release_date           9099 non-null   object 
 15  revenue 

# Tiếp tục tiền xử lý dữ liệu, và chọn lọc features cần thiết thôi

In [12]:
smd = smd[['id','title','overview',
        'tagline','genres','vote_count',
        'vote_average','popularity','release_date']]
smd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9099 entries, 0 to 45265
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9099 non-null   int32  
 1   title         9099 non-null   object 
 2   overview      9087 non-null   object 
 3   tagline       7033 non-null   object 
 4   genres        9099 non-null   object 
 5   vote_count    9099 non-null   float64
 6   vote_average  9099 non-null   float64
 7   popularity    9099 non-null   object 
 8   release_date  9099 non-null   object 
dtypes: float64(2), int32(1), object(6)
memory usage: 675.3+ KB


In [13]:
smd.isnull().sum()

id                 0
title              0
overview          12
tagline         2066
genres             0
vote_count         0
vote_average       0
popularity         0
release_date       0
dtype: int64

Ở phần overview, có 12 giá trị null, để đảm bảo chất lượng cho mô hình ở giai đoạn này, chúng ta sẽ ưu tiêu bỏ nó đi, còn tagline thì không cần bỏ, overview quan trọng hơn.

In [14]:
smd[smd['overview'].isnull()].transpose()

,300,641,821,1389,2194,14123,18393,26132,31657,34619,34796,34976
id,161495,10801,48144,43775,134368,140887,145594,3539,284362,332534,265351,295748
title,Roommates,The Superwife,The Day the Sun Turned Cold,Guantanamera,One Tough Cop,The Three Musketeers,Descongelate!,Off Beat,El vals de los inútiles,Bana Masal Anlatma,İtirazım Var,Pek Yakında
overview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tagline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
genres,"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'id': 35, 'name': 'Comedy'}]",[],"[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...",[],"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 18, 'name': 'Drama'}, {'id': 35, 'name...","[{'id': 18, 'name': 'Drama'}]","[{'id': 99, 'name': 'Documentary'}]","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam..."
vote_count,7.0,7.0,2.0,3.0,3.0,2.0,0.0,16.0,3.0,9.0,9.0,26.0
vote_average,6.4,5.3,7.0,8.0,3.0,3.0,0.0,6.5,5.7,5.7,7.1,7.1
popularity,3.395867,0.821299,0.076062,0.496315,1.082,0.694665,0.007517,1.149831,0.120001,1.095345,0.465766,2.088729
release_date,1995-03-01,1996-03-06,1994-01-01,1995-01-01,1998-10-09,1933-04-07,2003-09-12,2004-09-11,2014-05-08,2015-01-09,2014-04-18,2014-10-02


In [15]:
smd = smd.drop([300, 641, 821, 1389, 2194, 14123, 18393, 26132, 31657, 34619, 34796, 34976])

In [16]:
duplicate_rows = smd[smd.duplicated(subset=['id'], keep='first')]
# Lấy danh sách các id trùng lặp
duplicate_ids = duplicate_rows['id'].unique()
# Loại bỏ các dòng bị lặp từ DataFrame
smd = smd.drop_duplicates(subset=['id'], keep=False)
# In ra số hàng của DataFrame đã làm sạch
smd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9053 entries, 0 to 40503
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9053 non-null   int32  
 1   title         9053 non-null   object 
 2   overview      9053 non-null   object 
 3   tagline       7021 non-null   object 
 4   genres        9053 non-null   object 
 5   vote_count    9053 non-null   float64
 6   vote_average  9053 non-null   float64
 7   popularity    9053 non-null   object 
 8   release_date  9053 non-null   object 
dtypes: float64(2), int32(1), object(6)
memory usage: 671.9+ KB


In [17]:
smd.transpose()

,0,1,2,3,4,5,6,7,8,9,...,39619,39663,39669,39752,39771,39784,40004,40058,40224,40503
id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,373348,338766,390734,314420,390989,159550,392572,402672,315011,391698
title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Author: The JT LeRoy Story,Hell or High Water,Kingsglaive: Final Fantasy XV,Body,Sharknado 4: The 4th Awakens,The Last Brickmaker in America,Rustom,Mohenjo Daro,Shin Godzilla,The Beatles: Eight Days a Week - The Touring Y...
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...,...,New York magazine’s October 2005 issue sent sh...,A divorced dad and his ex-con brother resort t...,The magical kingdom of Lucis is home to the wo...,A night out turns deadly when three girls brea...,The new installment of the Sharknado franchise...,A man must cope with the loss of his wife and ...,"Rustom Pavri, an honourable officer of the Ind...","Village lad Sarman is drawn to big, bad Mohenj...",From the mind behind Evangelion comes a hit la...,"The band stormed Europe in 1963, and, in 1964,..."
tagline,NaN,Roll the dice and unleash the excitement!,Still Yelling. Still Fighting. Still Ready for...,Friends are the people who let you be yourself...,Just When His World Is Back To Normal... He's ...,A Los Angeles Crime Saga,You are cordially invited to the most surprisi...,The Original Bad Boys.,Terror goes into overtime.,No limits. No fears. No substitutes.,...,NaN,Blood always follows money.,Kingsglaive: Final Fantasy XV,NaN,"What happens in Vegas, stays in Vegas. Unless ...",NaN,Decorated Officer. Devoted Family Man. Defendi...,NaN,A god incarnate. A city doomed.,The band you know. The story you don't.
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 35, 'name': 'Comedy'}]","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",...,"[{'id': 99, 'name': 'Documentary'}]","[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[{'id': 28, 'name': 'Action'}, {'id': 16, 'nam...","[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...","[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...","[{'id': 18, 'name': 'Drama'}]","[{'id': 53, 'name': 'Thriller'}, {'id': 10749,...","[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...","[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[{'id': 99, 'name': 'Documentary'}, {'id': 104..."
vote_count,5415.0,2413.0,92.0,34.0,173.0,1886.0,141.0,45.0,174.0,1194.0,...,9.0,1304.0,201.0,16.0,88.0,1.0,25.0,26.0,152.0,92.0
vote_average,7.7,6.9,6.5,6.1,5.7,7.7,6.2,5.4,5.5,6.6,...,6.2,7.2,6.8,5.0,4.3,7.0,7.3,6.7,6.6,7.6
popularity,21.946943,17.015539,11.7129,3.859495,8.387519,17.924927,6.677277,2.561161,5.23158,14.686036,...,0.428899,12.565896,7.692445,6.236714,4.574494,0.038998,7.333139,1.423358,9.285519,7.078301
release_date,1995-10-30,1995-12-15,1995-12-22,1995-12-22,1995-02-10,1995-12-15,1995-12-15,1995-12-22,1995-12-22,1995-11-16,...,2016-01-22,2016-08-12,2016-07-09,2015-01-25,2016-07-31,2001-09-23,2016-08-12,2016-08-11,2016-07-29,2016-09-15


In [18]:
from ast import literal_eval
#Tiền xử lý cột genres (thể loại phim)
#Thay thế các giá trị NaN trong cột 'genres' bằng chuỗi '[]'.
#Còn các chuỗi Json được chuyển thành các đối tượng Python(dictionary).
smd['genres'] = smd['genres'].fillna('[]').apply(literal_eval)
#Trích xuất giá trị của khóa 'name' từ dictionary về kiểu danh sách (list)
smd['genres'] = smd['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['genres']

0                              [Animation, Comedy, Family]
1                             [Adventure, Fantasy, Family]
2                                        [Romance, Comedy]
3                                 [Comedy, Drama, Romance]
4                                                 [Comedy]
                               ...                        
39784                                              [Drama]
40004                                  [Thriller, Romance]
40058                 [Adventure, Drama, History, Romance]
40224    [Action, Adventure, Drama, Horror, Science Fic...
40503                                 [Documentary, Music]
Name: genres, Length: 9053, dtype: object

In [19]:
# Tiền xử lý dữ liệu để lấy được ra được thông tin year bằng split bằng '-'

smd['year'] = pd.to_datetime(smd['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)


In [20]:
smd = smd.drop(columns=['release_date'])

In [21]:
smd.transpose()

,0,1,2,3,4,5,6,7,8,9,...,39619,39663,39669,39752,39771,39784,40004,40058,40224,40503
id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,373348,338766,390734,314420,390989,159550,392572,402672,315011,391698
title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Author: The JT LeRoy Story,Hell or High Water,Kingsglaive: Final Fantasy XV,Body,Sharknado 4: The 4th Awakens,The Last Brickmaker in America,Rustom,Mohenjo Daro,Shin Godzilla,The Beatles: Eight Days a Week - The Touring Y...
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...,...,New York magazine’s October 2005 issue sent sh...,A divorced dad and his ex-con brother resort t...,The magical kingdom of Lucis is home to the wo...,A night out turns deadly when three girls brea...,The new installment of the Sharknado franchise...,A man must cope with the loss of his wife and ...,"Rustom Pavri, an honourable officer of the Ind...","Village lad Sarman is drawn to big, bad Mohenj...",From the mind behind Evangelion comes a hit la...,"The band stormed Europe in 1963, and, in 1964,..."
tagline,NaN,Roll the dice and unleash the excitement!,Still Yelling. Still Fighting. Still Ready for...,Friends are the people who let you be yourself...,Just When His World Is Back To Normal... He's ...,A Los Angeles Crime Saga,You are cordially invited to the most surprisi...,The Original Bad Boys.,Terror goes into overtime.,No limits. No fears. No substitutes.,...,NaN,Blood always follows money.,Kingsglaive: Final Fantasy XV,NaN,"What happens in Vegas, stays in Vegas. Unless ...",NaN,Decorated Officer. Devoted Family Man. Defendi...,NaN,A god incarnate. A city doomed.,The band you know. The story you don't.
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy],"[Action, Crime, Drama, Thriller]","[Comedy, Romance]","[Action, Adventure, Drama, Family]","[Action, Adventure, Thriller]","[Adventure, Action, Thriller]",...,[Documentary],"[Crime, Drama, Thriller, Western]","[Action, Animation, Adventure, Drama, Fantasy,...","[Drama, Thriller]","[Comedy, Horror, Science Fiction]",[Drama],"[Thriller, Romance]","[Adventure, Drama, History, Romance]","[Action, Adventure, Drama, Horror, Science Fic...","[Documentary, Music]"
vote_count,5415.0,2413.0,92.0,34.0,173.0,1886.0,141.0,45.0,174.0,1194.0,...,9.0,1304.0,201.0,16.0,88.0,1.0,25.0,26.0,152.0,92.0
vote_average,7.7,6.9,6.5,6.1,5.7,7.7,6.2,5.4,5.5,6.6,...,6.2,7.2,6.8,5.0,4.3,7.0,7.3,6.7,6.6,7.6
popularity,21.946943,17.015539,11.7129,3.859495,8.387519,17.924927,6.677277,2.561161,5.23158,14.686036,...,0.428899,12.565896,7.692445,6.236714,4.574494,0.038998,7.333139,1.423358,9.285519,7.078301
year,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,...,2016,2016,2016,2015,2016,2001,2016,2016,2016,2016


# Mô hình gợi ý phim dựa trên nội dung(1): Sử dụng mô tả và tagline của phim

## Trước tiên, chúng ta muốn xây dựng hệ thống mà sử dụng mô tả (overview + taglines)

`CountVectorizer` và `TfidfVectorizer` là hai công cụ phổ biến trong xử lý văn bản trong Machine Learning và Natural Language Processing (NLP). Dưới đây là sự khác biệt chính giữa chúng:

1. **CountVectorizer**:
   - Chuyển đổi một tập hợp các văn bản thành ma trận số lần xuất hiện của các từ trong các văn bản đó (tần suất xuất hiện).
   - Giá trị của mỗi ô trong ma trận là số lần xuất hiện của từ trong văn bản tương ứng.
   - Không xử lý trọng số hoặc tần suất xuất hiện của từng từ trong văn bản.

2. **TfidfVectorizer** (Term Frequency-Inverse Document Frequency):
   - Tính toán trọng số của mỗi từ trong một tập hợp các văn bản dựa trên tần suất xuất hiện của từ đó trong mỗi văn bản cộng với mức độ quan trọng của từ đó trong tất cả các văn bản.
   - Cân nhắc tần suất xuất hiện của từ trong một văn bản cụ thể (TF), cùng với tần suất xuất hiện của từ đó trong toàn bộ tập hợp các văn bản (IDF).
   - Giá trị của mỗi ô trong ma trận là giá trị TF-IDF của từng từ trong văn bản.

Hãy giải thích các tham số trong `TfidfVectorizer`:

1. **analyzer='word'**: Tham số này chỉ định phương pháp phân tích các đơn vị văn bản. Trong trường hợp này, 'word' chỉ ra rằng các đơn vị văn bản được xem xét là các từ riêng lẻ.

2. **ngram_range=(1, 2)**: Tham số này chỉ định phạm vi của các n-gram mà bạn muốn sử dụng. Trong trường hợp này, `(1, 2)` chỉ ra rằng các unigram (1-gram) và bigram (2-gram) sẽ được xem xét.

3. **min_df=0.0000000001**: Tham số này chỉ định ngưỡng tần suất tối thiểu của một từ để được tính trong vector của văn bản. Trong trường hợp này, giá trị là 0.0000000001, tức là bất kỳ từ nào xuất hiện ít nhất một lần trong toàn bộ tập dữ liệu sẽ được tính vào vector.

4. **stop_words='english'**: Tham số này chỉ định danh sách các từ dừng (stop words) mà bạn muốn loại bỏ khỏi văn bản. Trong trường hợp này, 'english' chỉ ra rằng danh sách từ dừng được sử dụng là danh sách từ dừng tiếng Anh được tích hợp sẵn trong scikit-learn.

Về câu hỏi vì sao `min_df` không được đặt bằng 0, điều này là do `min_df` xác định ngưỡng tần suất tối thiểu của một từ để được tính trong vector của văn bản. Nếu đặt `min_df=0`, nó có nghĩa là mọi từ đều được tính vào vector, bất kể tần suất xuất hiện của chúng. Điều này có thể dẫn đến các từ rất phổ biến và ít thông tin (ví dụ: từ dừng) chiếm quá nhiều không gian trong vector và ảnh hưởng đến hiệu suất của mô hình. Để tránh điều này, chúng ta thường đặt `min_df` thành một giá trị nhỏ, như `min_df=0.0000000001`, như trong trường hợp này.

In [22]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0000000001, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [24]:
tfidf_matrix.shape

(9053, 267555)


1. **Sử dụng TF-IDF Vectorizer**: Khi sử dụng `TF-IDF Vectorizer` để biểu diễn các văn bản, chúng ta thu được một ma trận các vectơ, trong đó mỗi vectơ biểu diễn một văn bản và mỗi phần tử trong vectơ biểu diễn một từ hoặc n-gram và có giá trị tương ứng với trọng số TF-IDF của từ hoặc n-gram đó trong văn bản.

2. **Tính toán Độ tương đồng Cosine**: Độ tương đồng cosine là một phép đo độ tương đồng giữa hai vectơ trong không gian nhiều chiều. Trong trường hợp này, các vectơ biểu diễn các văn bản trong không gian các từ hoặc n-gram. Độ tương đồng cosine giữa hai văn bản được tính bằng cách lấy tích vô hướng của hai vectơ biểu diễn hai văn bản và chia cho tích của độ dài của hai vectơ đó.

3. **Sử dụng `linear_kernel` thay vì `cosine_similarities`**: `linear_kernel` là một hàm trong scikit-learn được sử dụng để tính toán tích vô hướng giữa các vectơ. Trong trường hợp này, chúng ta không cần phải sử dụng `cosine_similarities` để tính toán độ tương đồng cosine, mà có thể sử dụng trực tiếp `linear_kernel`. Việc này tiết kiệm thời gian tính toán vì `linear_kernel` được cài đặt để tận dụng hiệu quả các tính chất của tích vô hướng trong không gian vector.

Vì vậy, ý nghĩa của câu này là khi sử dụng `TF-IDF Vectorizer` để biểu diễn văn bản, việc tính toán độ tương đồng cosine giữa các văn bản có thể được thực hiện một cách nhanh chóng bằng cách sử dụng `linear_kernel` thay vì `cosine_similarities`.

In [25]:
# http://scikit-learn.org/stable/modules/metrics.html#linear-kernel
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [26]:
cosine_sim[0:5]

array([[1.        , 0.00680015, 0.        , ..., 0.        , 0.        ,
        0.00477555],
       [0.00680015, 1.        , 0.01530131, ..., 0.        , 0.00175585,
        0.0036893 ],
       [0.        , 0.01530131, 1.        , ..., 0.0019266 , 0.00221451,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00224727, 0.00789369, ..., 0.        , 0.00380764,
        0.0018776 ]])

* Giờ chúng ta đã có được ma trận độ tương đồng giữa tất cả các cặp phim trong dataset.
* Tiếp theo là viết 1 hàm để trả về 30 bộ phim gần nhất cho 1 bộ phim cụ thể, dựa trên điểm tương đồng cosine.

In [27]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [28]:
smd = smd.drop(columns=['index'])

In [29]:
smd.transpose()

,0,1,2,3,4,5,6,7,8,9,...,9043,9044,9045,9046,9047,9048,9049,9050,9051,9052
id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,373348,338766,390734,314420,390989,159550,392572,402672,315011,391698
title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Author: The JT LeRoy Story,Hell or High Water,Kingsglaive: Final Fantasy XV,Body,Sharknado 4: The 4th Awakens,The Last Brickmaker in America,Rustom,Mohenjo Daro,Shin Godzilla,The Beatles: Eight Days a Week - The Touring Y...
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...,...,New York magazine’s October 2005 issue sent sh...,A divorced dad and his ex-con brother resort t...,The magical kingdom of Lucis is home to the wo...,A night out turns deadly when three girls brea...,The new installment of the Sharknado franchise...,A man must cope with the loss of his wife and ...,"Rustom Pavri, an honourable officer of the Ind...","Village lad Sarman is drawn to big, bad Mohenj...",From the mind behind Evangelion comes a hit la...,"The band stormed Europe in 1963, and, in 1964,..."
tagline,,Roll the dice and unleash the excitement!,Still Yelling. Still Fighting. Still Ready for...,Friends are the people who let you be yourself...,Just When His World Is Back To Normal... He's ...,A Los Angeles Crime Saga,You are cordially invited to the most surprisi...,The Original Bad Boys.,Terror goes into overtime.,No limits. No fears. No substitutes.,...,,Blood always follows money.,Kingsglaive: Final Fantasy XV,,"What happens in Vegas, stays in Vegas. Unless ...",,Decorated Officer. Devoted Family Man. Defendi...,,A god incarnate. A city doomed.,The band you know. The story you don't.
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy],"[Action, Crime, Drama, Thriller]","[Comedy, Romance]","[Action, Adventure, Drama, Family]","[Action, Adventure, Thriller]","[Adventure, Action, Thriller]",...,[Documentary],"[Crime, Drama, Thriller, Western]","[Action, Animation, Adventure, Drama, Fantasy,...","[Drama, Thriller]","[Comedy, Horror, Science Fiction]",[Drama],"[Thriller, Romance]","[Adventure, Drama, History, Romance]","[Action, Adventure, Drama, Horror, Science Fic...","[Documentary, Music]"
vote_count,5415.0,2413.0,92.0,34.0,173.0,1886.0,141.0,45.0,174.0,1194.0,...,9.0,1304.0,201.0,16.0,88.0,1.0,25.0,26.0,152.0,92.0
vote_average,7.7,6.9,6.5,6.1,5.7,7.7,6.2,5.4,5.5,6.6,...,6.2,7.2,6.8,5.0,4.3,7.0,7.3,6.7,6.6,7.6
popularity,21.946943,17.015539,11.7129,3.859495,8.387519,17.924927,6.677277,2.561161,5.23158,14.686036,...,0.428899,12.565896,7.692445,6.236714,4.574494,0.038998,7.333139,1.423358,9.285519,7.078301
year,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,...,2016,2016,2016,2015,2016,2001,2016,2016,2016,2016
description,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...,...,New York magazine’s October 2005 issue sent sh...,A divorced dad and his ex-con brother resort t...,The magical kingdom of Lucis is home to the wo...,A night o

In [30]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

* Đã xong kiểu mẫu ban đầu!
* Giờ chúng ta sẽ thử đưa ra một vài yêu cầu để xem mô hình hoạt động đến đâu

In [31]:
get_recommendations('Wolf Children').head(10)

3178                         Bounce
1729    I Married a Strange Person!
6743    I Could Never Be Your Woman
2873                          Loser
2804                     Parenthood
3044             Autumn in New York
5966                In Good Company
5358                        College
4372                  About Schmidt
8920              The Men Next Door
Name: title, dtype: object

In [32]:
get_recommendations('Castle in the Sky').head(10)

1687                         The Dark Crystal
8325           Percy Jackson: Sea of Monsters
5553    Sky Captain and the World of Tomorrow
6154                                  Stealth
652                                Phenomenon
115                            Pie in the Sky
6249                           Chicken Little
5114                        Home on the Range
7062                                     Igor
2829                      The Virgin Suicides
Name: title, dtype: object

In [33]:
get_recommendations('Finding Nemo').head(10)

9016                              Finding Dory
5569                                Shark Tale
2732          Teenage Mutant Ninja Turtles III
4233                 The Incredible Mr. Limpet
7544                                Mr. Nobody
2702                              Turtle Diary
814               20,000 Leagues Under the Sea
5348                             Arizona Dream
2232    Little Nemo: Adventures In Slumberland
2568              Fast Times at Ridgemont High
Name: title, dtype: object

In [34]:
get_recommendations('Neon Genesis Evangelion: Death and Rebirth').head(10)

8777                                  Ismael
2766                                   Lucas
390                           Demolition Man
5913                            Lost Embrace
8412      Evangelion: 3.0 You Can (Not) Redo
7927                   Underworld: Awakening
7255    Evangelion: 1.0: You Are (Not) Alone
3014                                  Shower
6343                             Ultraviolet
5462                           Before Sunset
Name: title, dtype: object

In [35]:
get_recommendations('My Neighbor Totoro').head(10)

3494    Final Fantasy: The Spirits Within
7719                      Red Riding Hood
5733                         The Landlord
8786                          Poltergeist
7730                            Insidious
4937                 What's New Pussycat?
8171                      Our Little Girl
3776                       Female Trouble
1588             The Cat from Outer Space
1212                         Fathers' Day
Name: title, dtype: object

In [36]:
get_recommendations('Spirited Away').head(10)

450                                  North
3744    I Never Promised You a Rose Garden
674                                Matilda
6858             The Spiderwick Chronicles
6383                            Hard Candy
6213                            Mirrormask
3852             Jimmy Neutron: Boy Genius
8853       Zenon: Girl of the 21st Century
648                               Daylight
4585                    Friends and Family
Name: title, dtype: object

In [37]:
get_recommendations('The Godfather').head(10)

967      The Godfather: Part II
8359                 The Family
3499                       Made
4184         Johnny Dangerously
29               Shanghai Triad
5651                       Fury
2403             American Movie
1574    The Godfather: Part III
4209                    8 Women
2150              Summer of Sam
Name: title, dtype: object

**Kết quả:** Kết quả không mấy tự hào, như dự đoán, bởi vì chỉ kết hợp overview và tagline, nên kết quả cho ra chỉ có ích đối với 1 số phim có cùng series 

## Mô hình gợi ý phim dựa trên nội dung (2): 
Từ kết quả đã đạt được, có thể thấy, nó còn quá đơn sơ và chưa thể sử dụng.
Chính vì thế, tiếp theo chúng ta cần kết hợp bộ dữ liệu `movies_metadata.csv` với các bộ dữ liệu khác `keywords.csv` và `credits.csv` để lấy keywords, diễn viên chính, và đạo diễn, làm feature cho huấn luyện mô hình.

In [38]:
keywords = pd.read_csv('input_data/keywords.csv')
credits =pd.read_csv('input_data/credits.csv')

In [39]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')

In [40]:
smd.shape

(9053, 10)

In [41]:
smd = smd.merge(credits, on='id')
smd = smd.merge(keywords, on ='id')
smd.shape

(9071, 13)

In [42]:
smd.isnull().sum()

id              0
title           0
overview        0
tagline         0
genres          0
vote_count      0
vote_average    0
popularity      0
year            0
description     0
cast            0
crew            0
keywords        0
dtype: int64

In [43]:
smd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9071 entries, 0 to 9070
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9071 non-null   int32  
 1   title         9071 non-null   object 
 2   overview      9071 non-null   object 
 3   tagline       9071 non-null   object 
 4   genres        9071 non-null   object 
 5   vote_count    9071 non-null   float64
 6   vote_average  9071 non-null   float64
 7   popularity    9071 non-null   object 
 8   year          9071 non-null   object 
 9   description   9071 non-null   object 
 10  cast          9071 non-null   object 
 11  crew          9071 non-null   object 
 12  keywords      9071 non-null   object 
dtypes: float64(2), int32(1), object(10)
memory usage: 886.0+ KB


Thấy rằng số hàng là 9071, nhiều hơn so với trước khi merge, vậy nên chắc chắn có trùng lặp ID. Chúng ta sẽ clean lại như sau

In [44]:
# Xác định các dòng bị lặp dựa trên cột 'id'
duplicate_rows = smd[smd.duplicated(subset=['id'], keep='first')]
# Lấy danh sách các id trùng lặp
duplicate_ids = duplicate_rows['id'].unique()
# Loại bỏ các dòng bị lặp từ DataFrame
smd = smd.drop_duplicates(subset=['id'], keep=False)
# In ra số hàng của DataFrame đã làm sạch
smd.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9035 entries, 0 to 9070
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            9035 non-null   int32  
 1   title         9035 non-null   object 
 2   overview      9035 non-null   object 
 3   tagline       9035 non-null   object 
 4   genres        9035 non-null   object 
 5   vote_count    9035 non-null   float64
 6   vote_average  9035 non-null   float64
 7   popularity    9035 non-null   object 
 8   year          9035 non-null   object 
 9   description   9035 non-null   object 
 10  cast          9035 non-null   object 
 11  crew          9035 non-null   object 
 12  keywords      9035 non-null   object 
dtypes: float64(2), int32(1), object(10)
memory usage: 952.9+ KB


Bây giờ ta đã có diễn viên chính, thể loại, đoàn làm phim trong cùng 1 dataframe:

**1. Đoàn làm phim:** Chủ yếu quan tâm đến đạo diễn, nên chúng ta sẽ làm một chút tiền xử lý.

**2. Diễn viên chính:** Chúng ta sẽ chỉ mong muốn lấy ra những diễn viên chính, mỗi bộ phim có số lượng diễn viên chính (hoặc phụ, nhưng có vai trò lớn) không xác định được, nhưng có lẽ con số 3 sẽ còn số phù hợp, chúng ta sẽ lấy ra 3 diễn viên đầu trong từ điển.

In [45]:
#tương tự bước trên, literal_eval để đưa các trường này về dạng dictionary chính thức
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [46]:
smd['crew_size'].value_counts().sort_index()

crew_size
0       14
1      276
2      548
3      510
4      311
      ... 
228      1
242      1
244      1
338      1
435      1
Name: count, Length: 178, dtype: int64

In [47]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [48]:
smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [49]:
smd['cast'].iloc[0:3]

0               [Tom Hanks, Tim Allen, Don Rickles]
1    [Robin Williams, Jonathan Hyde, Kirsten Dunst]
2        [Walter Matthau, Jack Lemmon, Ann-Margret]
Name: cast, dtype: object

In [50]:
smd['keywords'].iloc[0:5]

0    [jealousy, toy, boy, friendship, friends, riva...
1    [board game, disappearance, based on children'...
2    [fishing, best friend, duringcreditsstinger, o...
3    [based on novel, interracial relationship, sin...
4    [baby, midlife crisis, confidence, aging, daug...
Name: keywords, dtype: object

In [51]:
smd['director'].iloc[0:5]

0      John Lasseter
1       Joe Johnston
2      Howard Deutch
3    Forest Whitaker
4      Charles Shyer
Name: director, dtype: object

In [52]:
smd['genres'].iloc[0:5]

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

* Để mô hình hoạt động tốt, sẽ tương đối hack não.

* Ở đây, chúng ta sẽ tạo ra 1 "thập cẩm" cho mỗi phim, trong đó chứa thể loại, đạo diễn, diễn viên chính, và từ khoá.

* Sau đó sử dụng __Count Vectorizer__ để tạo 1 __count matrix__

* Các bước còn lại sẽ tương tự như đã làm ở trước đó.

Các bước chuẩn bị cho nồi "thập cẩm" sẽ tiến hành như sau:

1. __Loại bỏ khoảng trống và uppercase__ từ tất cả các feature. Để đảm bảo mô hình không nhầm lẫn giữa __Johnny Depp and Johnny Galecki.__
2. __Tăng trọng số cho đạo diễn__ bằng cách __lặp lại nó 3 lần__ để nó tương đồng với số diễn viên ta đã chọn, từ đó, trọng số của đạo diễn cũng quan trọng ngang diễn viên chính

In [53]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x, x, x])

In [54]:
smd['director']

0              [johnlasseter, johnlasseter, johnlasseter]
1                 [joejohnston, joejohnston, joejohnston]
2              [howarddeutch, howarddeutch, howarddeutch]
3        [forestwhitaker, forestwhitaker, forestwhitaker]
4              [charlesshyer, charlesshyer, charlesshyer]
                              ...                        
9066        [greggchampion, greggchampion, greggchampion]
9067    [tinusureshdesai, tinusureshdesai, tinusureshd...
9068    [ashutoshgowariker, ashutoshgowariker, ashutos...
9069              [hideakianno, hideakianno, hideakianno]
9070                    [ronhoward, ronhoward, ronhoward]
Name: director, Length: 9035, dtype: object

**Keywords**

* Chúng ta sẽ tiền xử lý một chút trước khi đưa vào sử dụng nó
* Đầu tiên __tính tần xuất xuất hiện của mỗi từ__ 

In [55]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s

keyword
independent film        602
woman director          538
murder                  395
duringcreditsstinger    327
based on novel          306
                       ... 
driving test              1
derbyshire                1
worm                      1
joy                       1
toyko                     1
Name: count, Length: 12905, dtype: int64

* Keywords có tần số trải từ 1 đến 610.
* Và chúng ta chắc chắn không cần những keywords chỉ xuất hiện 1 lần
* Cuối cùng thì, chúng ta sẽ bình thường hoá các từ về dạng gốc của nó VD: __Dogs__ sẽ đưa về __Dog__.

In [56]:
s = s[s > 1]

In [57]:
from nltk.stem.snowball import SnowballStemmer
# Just an example
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [58]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [59]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [60]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))
smd['soup'].iloc[0:5]

0    jealousi toy boy friendship friend rivalri boy...
1    boardgam disappear basedonchildren'sbook newho...
2    fish bestfriend duringcreditssting waltermatth...
3    basedonnovel interracialrelationship singlemot...
4    babi midlifecrisi confid age daughter motherda...
Name: soup, dtype: object

In [61]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0.0000000001, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [62]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim[0]

array([1.        , 0.02441931, 0.02738955, ..., 0.        , 0.        ,
       0.        ])

In [63]:
#đánh lại chỉ mục cho dataframe
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [64]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9030
Rustom                                                9031
Mohenjo Daro                                          9032
Shin Godzilla                                         9033
The Beatles: Eight Days a Week - The Touring Years    9034
Length: 9035, dtype: int64

In [65]:
smd = smd.drop(columns=['index'])

In [66]:
smd.transpose()

,0,1,2,3,4,5,6,7,8,9,...,9025,9026,9027,9028,9029,9030,9031,9032,9033,9034
id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,373348,338766,390734,314420,390989,159550,392572,402672,315011,391698
title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye,...,Author: The JT LeRoy Story,Hell or High Water,Kingsglaive: Final Fantasy XV,Body,Sharknado 4: The 4th Awakens,The Last Brickmaker in America,Rustom,Mohenjo Daro,Shin Godzilla,The Beatles: Eight Days a Week - The Touring Y...
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...,...,New York magazine’s October 2005 issue sent sh...,A divorced dad and his ex-con brother resort t...,The magical kingdom of Lucis is home to the wo...,A night out turns deadly when three girls brea...,The new installment of the Sharknado franchise...,A man must cope with the loss of his wife and ...,"Rustom Pavri, an honourable officer of the Ind...","Village lad Sarman is drawn to big, bad Mohenj...",From the mind behind Evangelion comes a hit la...,"The band stormed Europe in 1963, and, in 1964,..."
tagline,,Roll the dice and unleash the excitement!,Still Yelling. Still Fighting. Still Ready for...,Friends are the people who let you be yourself...,Just When His World Is Back To Normal... He's ...,A Los Angeles Crime Saga,You are cordially invited to the most surprisi...,The Original Bad Boys.,Terror goes into overtime.,No limits. No fears. No substitutes.,...,,Blood always follows money.,Kingsglaive: Final Fantasy XV,,"What happens in Vegas, stays in Vegas. Unless ...",,Decorated Officer. Devoted Family Man. Defendi...,,A god incarnate. A city doomed.,The band you know. The story you don't.
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy],"[Action, Crime, Drama, Thriller]","[Comedy, Romance]","[Action, Adventure, Drama, Family]","[Action, Adventure, Thriller]","[Adventure, Action, Thriller]",...,[Documentary],"[Crime, Drama, Thriller, Western]","[Action, Animation, Adventure, Drama, Fantasy,...","[Drama, Thriller]","[Comedy, Horror, Science Fiction]",[Drama],"[Thriller, Romance]","[Adventure, Drama, History, Romance]","[Action, Adventure, Drama, Horror, Science Fic...","[Documentary, Music]"
vote_count,5415.0,2413.0,92.0,34.0,173.0,1886.0,141.0,45.0,174.0,1194.0,...,9.0,1304.0,201.0,16.0,88.0,1.0,25.0,26.0,152.0,92.0
vote_average,7.7,6.9,6.5,6.1,5.7,7.7,6.2,5.4,5.5,6.6,...,6.2,7.2,6.8,5.0,4.3,7.0,7.3,6.7,6.6,7.6
popularity,21.946943,17.015539,11.7129,3.859495,8.387519,17.924927,6.677277,2.561161,5.23158,14.686036,...,0.428899,12.565896,7.692445,6.236714,4.574494,0.038998,7.333139,1.423358,9.285519,7.078301
year,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,...,2016,2016,2016,2015,2016,2001,2016,2016,2016,2016
description,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...,...,New York magazine’s October 2005 issue sent sh...,A divorced dad and his ex-con brother resort t...,The magical kingdom of Lucis is home to the wo...,A night o

* Tái sử dụng hàm get_recommendations đã viết trước đó. 
* Điểm tương đồng cosine lần này đã khác nên chắc chắn kết quả cũng khác, nhưng chúng ta sẽ mong muốn nó hoạt động tốt hơn trước.


In [67]:
get_recommendations('Wolf Children').head(10)

7712                        Summer Wars
8966              The Boy and the Beast
6842    The Girl Who Leapt Through Time
3140                 Digimon: The Movie
2291                         Thumbelina
1569                              Bambi
215                               Gordy
574                           Space Jam
1686                 The Secret of NIMH
1441                  Quest for Camelot
Name: title, dtype: object

Bùm, kết quả hiện tại, ở lần thử đầu tiên đã ấn tượng hơn rất nhiều so với trước
* Cho ra được các phim cùng thể loại Animation
* Và cùng đạo diễn Mamoru Hosoda

In [68]:
get_recommendations('Toy Story').head(10)

6293                Luxo Jr.
2493             Toy Story 2
7788                  Cars 2
6403                    Cars
1854            A Bug's Life
8384    Toy Story of Terror!
7511             Toy Story 3
2722       Creature Comforts
3788          Monsters, Inc.
1404        Meet the Deedles
Name: title, dtype: object

In [69]:
get_recommendations('The Dark Knight').head(10)

7905         The Dark Knight Rises
6125                 Batman Begins
6522                  The Prestige
2056                     Following
7530                     Inception
4092                      Insomnia
3352                       Memento
8478                  Interstellar
7541    Batman: Under the Red Hood
1107                Batman Returns
Name: title, dtype: object

Kết quả đối với phim có diễn viên đóng (thay vì diễn viên lồng tiếng như Animation) cũng khá oke.

**Cải tiếp** 

* Chúng ta hoàn toàn có thể thử thủ công các tham số, trọng số khác nha ucho mô hình hiện tại, hoặc giới hạn từ khoá >n thay vì >1, cân nhắc đến việc tần suất xuất hiện của các thể loại, vân vân.

# Mô hình gợi ý phim dựa trên nội dung (3): Thêm độ phổ biến và đánh giá trung bình vào tính toán trọng số

* Tuy các điểm đánh giá trung bình hay độ phổ biến không hoàn toàn khách quan nhưng nó vẫn có một số giá trị, chúng ta có thể đưa ra 2 sceneries như sau:
- Đối với phim (cụ thể là người đóng), ở trên Tmdb, các đánh giá này thường khách quan hơn vì có nhiều lượt đánh giá.
- Đối với những phim hoạt hình (Animation) thường không có nhiều đánh giá, và điểm đánh giá cũng không khách quan, khi đối tượng người xem phổ thông thường không đánh giá cao phim hoạt hình. Thay vào đó, những đánh giá về phim hoạt hình thường khách quan hơn ở những nơi chúng có tham chiếu lẫn nhau như __Myanimelist.net__

* Nhưng dù vậy, chúng ta vẫn nên tạo 1 cơ chế để loại bỏ những phim có đánh giá quá thấp mà sẽ trả về những phim tích cực được đón nhận hơn.


* Chúng ta sử dụng đánh giá của TMDB để tìm ra Top Các phim. 
* Thông qua 1 công thức tính trọng số như sau:

$\large Weighted\; Rating (WR) = (\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$
```
Trong đó,
    v số lượng đánh giá (votes) cho phim đó
    m là số lượng vote tối thiểu để bộ phim đạt tiêu chuẩn cho chart
    R là điểm số đánh giá của phim
    C là điểm số đánh giá trung bình cho cả tệp
```

In [70]:
vote_counts = smd[smd['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = smd[smd['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
m = vote_counts.quantile(0.60)

In [71]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [72]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & 
                       (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [73]:
improved_recommendations('Wolf Children')

,title,vote_count,vote_average,year,wr
2137,Tarzan,1715,7,1999,6.909922
828,The Fox and the Hound,884,7,1981,6.837947
6842,The Girl Who Leapt Through Time,435,7,2006,6.714830
5906,The Cat Returns,364,7,2002,6.675894
8966,The Boy and the Beast,304,7,2015,6.633619
7712,Summer Wars,265,7,2009,6.599679
1686,The Secret of NIMH,186,7,1982,6.507207
1569,Bambi,1450,6,1942,5.992195
574,Space Jam,1335,6,1996,5.991593
6441,Monster House,912,6,2006,5.988263


In [74]:
improved_recommendations('The Dark Knight')

,title,vote_count,vote_average,year,wr
7530,Inception,14075,8,2010,7.977195
8478,Interstellar,11187,8,2014,7.971389
6522,The Prestige,4510,8,2006,7.930447
3352,Memento,4168,8,2000,7.924946
7905,The Dark Knight Rises,9263,7,2012,6.982107
6125,Batman Begins,7511,7,2005,6.978018
7541,Batman: Under the Red Hood,459,7,2010,6.725959
2056,Following,363,7,1998,6.675269
7875,Batman: Year One,255,7,2011,6.589939
1107,Batman Returns,1706,6,1992,5.993268


Kết quả cho tới hiện tại là khá hài lòng, cảm nhận cá nhân của tôi cho thấy rằng chúng rất phù hợp với tôi. Tuy nhiên đó chưa phải tất cả, chúng ta sẽ sang phần tiếp theo để xây dựng một mô hình gợi ý dựa trên bộ lọc cộng tác

# Mô hình gợi ý dựa trên bộ lọc cộng tác (Collaborative Filtering)

**Mô hình gợi ý dựa trên nội dung có những nhược điểm chí mạng**

* Nó chỉ có thể gợi ý dựa trên 1 phim cụ thể.
* Không có tính cá nhân hoá cho từng user, nên nếu các user khác nhau thực hiện cùng 1 query, sẽ tả về kết quả hoàn toàn khác nhau.
* Tuy không thể nói bộ lọc cộng tác sẽ giải quyết hoàn toàn những nhược điểm trên, nhưng vẫn sẽ có tính cá nhân hoá lớn.



**Về bộ lọc cộng tác**

* Ý tưởng khái quát của nó là việc dựa trên những người dùng đã cùng xem phim đó, và họ cũng xem phim A, B, C để gợi ý cho người dùng khác.
* Chúng ta sẽ sử dụng thư viện Surpire, vì nó đã cung cấp thuật toán vô cùng mạnh mẽ như là __Singular Value Decomposition (SVD)__ để tối ưu  RMSE và cho ra dự đoán tốt nhất.


In [75]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [76]:
#đọc dữ liệu rating của từng user cụ thể
ratings = pd.read_csv('input_data/ratings_small.csv')

In [77]:
# surprise reader API to read the dataset
reader = Reader()

In [78]:
#không cần timestamp của tệp dữ liệu vì nó không liên quan
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [79]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5)

{'test_rmse': array([0.90150402, 0.89375051, 0.89965584, 0.89111546, 0.89234418]),
 'test_mae': array([0.69610446, 0.68684753, 0.69368492, 0.68666755, 0.68670042]),
 'fit_time': (3.295283079147339,
  2.4235494136810303,
  1.8990719318389893,
  2.694979190826416,
  1.7185406684875488),
 'test_time': (0.4420192241668701,
  0.21472954750061035,
  0.5035631656646729,
  0.281904935836792,
  0.14962100982666016)}

In [80]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [81]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=3.027959149677253, details={'was_impossible': False})

# Hybrid recommendation system

* In this section, will try to build a simple hybrid recommender that brings together techniques we have implemented in the content based and collaborative filter based engines. This is how it will work:

* **Input:** User ID and the Title of a Movie
* **Output:** Similar movies sorted on the basis of expected ratings by that particular user.

In [82]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [85]:
#mapping ID dựa trên tmdbID trong links_small.csv
id_map = pd.read_csv('input_data/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
id_map

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0
...,...,...
The Last Brickmaker in America,161944,159550.0
Rustom,162542,392572.0
Mohenjo Daro,162672,402672.0


In [86]:
indices_map = id_map.set_index('id')
indices_map

,movieId
id,
862.0,1
8844.0,2
15602.0,3
31357.0,4
11862.0,5
...,...
159550.0,161944
392572.0,162542
402672.0,162672


In [ ]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [ ]:
hybrid(1, 'Castle in the Sky')

,title,vote_count,vote_average,year,id,est
2396,Princess Mononoke,2041.0,8.2,1997,128,3.353484
6010,Howl's Moving Castle,2049.0,8.2,2004,4935,3.300827
4212,Spirited Away,3968.0,8.3,2001,129,3.275309
4391,My Neighbor Totoro,1730.0,8.0,1988,8392,3.247620
5830,Porco Rosso,563.0,7.6,1992,11621,3.093632
520,Aladdin,3495.0,7.4,1992,812,3.072665
4975,Nausicaä of the Valley of the Wind,808.0,7.7,1984,81,3.058591
5808,Kiki's Delivery Service,768.0,7.6,1989,16859,3.054984
7102,Ponyo,953.0,7.5,2008,12429,3.029469
8336,The Wind Rises,720.0,7.7,2013,149870,2.968567


In [ ]:
hybrid(1, 'The Girl Who Leapt Through Time')

,title,vote_count,vote_average,year,id,est
8342,About Time,2140.0,7.8,2013,122906,3.109470
5905,Ghost in the Shell 2: Innocence,230.0,7.3,2004,12140,3.006105
8254,Wolf Children,483.0,8.0,2012,110420,2.952218
3857,Little Otik,33.0,7.2,2001,18939,2.937799
7544,Mr. Nobody,1616.0,7.9,2009,31011,2.873858
1030,Somewhere in Time,104.0,7.1,1980,16633,2.841844
8671,Doctor Who: The Time of the Doctor,61.0,8.4,2013,282848,2.808269
5883,Blood: The Last Vampire,76.0,6.6,2000,919,2.791845
7712,Summer Wars,265.0,7.4,2009,28874,2.779248
5862,The End of Evangelion,124.0,8.1,1997,18491,2.751595


In [ ]:
import pickle

# Xuất các DataFrame ra tệp pickle
smd.to_pickle('AI_Dump/smd.pkl')
indices.to_pickle('AI_Dump/indices.pkl')
id_map.to_pickle('AI_Dump/id_map.pkl')
indices_map.to_pickle('AI_Dump/indices_map.pkl')

# Xuất ma trận cosine_sim ra tệp numpy
np.save('AI_Dump/cosine_sim.npy', cosine_sim)

NameError: name 'smd' is not defined

In [ ]:
with open('processed_data/svd_model.pkl', 'wb') as f:
    pickle.dump(svd, f)

In [ ]:
user_ids = ratings['userId'].unique()
np.save('AI_Dump/user_ids.npy', user_ids)